This notebook is used to create a csv file that includes my point projections in a format similar to fplreview.com. This csv file can then be used for making advanced team optimization plans using Sertalp B. Cay's repo "fpl-optimization".

In [1]:
latest_gameweek = 0

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
import json
import requests

pd.set_option('display.max_columns', None)

# Save team data
Info on your own team; starting point for the optimization.

In [3]:
# get login credentials for fetching team data
file_path = Path('../../login_credentials/fpl_login.json')
#file_path = Path('../../login_credentials/fpl_login2.json')
login_credentials = pd.read_json(file_path, typ='series')

In [4]:
# utility function for fetching team data
from src.utils import fetch_my_team

In [5]:
# fetch my team data
my_team = fetch_my_team(login_credentials.user_name, login_credentials.password, login_credentials.team_id)
# drop assistant manager if exists
my_team['picks'] = [item for item in my_team['picks'] if item.get('position') != 16]
my_team

KeyError: 'picks'

In [ ]:
my_team

In [ ]:
# save team data to fpl-optimization repo
file_path = Path('../../../fpl-optimization/data/team.json')
with open(file_path, 'w') as f:
    json.dump(my_team, f)

# Edit projections into fplreview format 
### (to be used with fpl-optimization repo solver)

In [6]:
filepath = Path(f'../data/predictions/gameweek{latest_gameweek}.csv')
projections = pd.read_csv(filepath, index_col=0)
if 'id' in projections.columns:
    projections = projections.drop('id', axis=1)
display(projections.head())
display(projections.shape)

,index,first_name,second_name,assists,bonus,bps,clean_sheets,corners_and_indirect_freekicks_order,creativity,creativity_rank,creativity_rank_type,direct_freekicks_order,dreamteam_count,element_type,event_points,goals_conceded,goals_scored,ict_index,ict_index_rank,ict_index_rank_type,influence,influence_rank,influence_rank_type,minutes,now_cost,own_goals,penalties_missed,penalties_order,penalties_saved,points_per_game,red_cards,saves,selected_by_percent,threat,threat_rank,threat_rank_type,total_points,web_name,yellow_cards,team_name,gameweek,season,gameweek_xG,gameweek_xA,gameweek_xGA,gameweek_minutes,team_xG,team_xGA,team_xG_ewm_5,team_xG_ewm_10,team_xG_ewm_20,team_xG_ewm_40,team_xGA_ewm_5,team_xGA_ewm_10,team_xGA_ewm_20,team_xGA_ewm_40,opponent_xG,opponent_xGA,opponent_xG_ewm_5,opponent_xG_ewm_10,opponent_xG_ewm_20,opponent_xG_ewm_40,opponent_xGA_ewm_5,opponent_xGA_ewm_10,opponent_xGA_ewm_20,opponent_xGA_ewm_40,home,gameweek_assists,gameweek_bps,gameweek_creativity,gameweek_goals_scored,gameweek_goals_conceded,gameweek_own_goals,gameweek_penalties_saved,gameweek_red_cards,gameweek_saves,gameweek_threat,gameweek_yellow_cards,gameweek_xPoints,gameweek_assists_ewm_5,gameweek_bps_ewm_5,gameweek_creativity_ewm_5,event_points_ewm_5,gameweek_goals_scored_ewm_5,gameweek_goals_conceded_ewm_5,gameweek_saves_ewm_5,gameweek_threat_ewm_5,gameweek_xG_ewm_5,gameweek_xA_ewm_5,gameweek_xGA_ewm_5,gameweek_minutes_ewm_5,gameweek_xPoints_ewm_5,gameweek_assists_ewm_10,gameweek_bps_ewm_10,gameweek_creativity_ewm_10,event_points_ewm_10,gameweek_goals_scored_ewm_10,gameweek_goals_conceded_ewm_10,gameweek_saves_ewm_10,gameweek_threat_ewm_10,gameweek_xG_ewm_10,gameweek_xA_ewm_10,gameweek_xGA_ewm_10,gameweek_minutes_ewm_10,gameweek_xPoints_ewm_10,gameweek_assists_ewm_20,gameweek_bps_ewm_20,gameweek_creativity_ewm_20,event_points_ewm_20,gameweek_goals_scored_ewm_20,gameweek_goals_conceded_ewm_20,gameweek_saves_ewm_20,gameweek_threat_ewm_20,gameweek_xG_ewm_20,gameweek_xA_ewm_20,gameweek_xGA_ewm_20,gameweek_minutes_ewm_20,gameweek_xPoints_ewm_20,gameweek_assists_ewm_40,gameweek_bps_ewm_40,gameweek_creativity_ewm_40,event_points_ewm_40,gameweek_goals_scored_ewm_40,gameweek_goals_conceded_ewm_40,gameweek_saves_ewm_40,gameweek_threat_ewm_40,gameweek_xG_ewm_40,gameweek_xA_ewm_40,gameweek_xGA_ewm_40,gameweek_minutes_ewm_40,gameweek_xPoints_ewm_40,gameweek_assists_expanding,gameweek_bps_expanding,gameweek_creativity_expanding,event_points_expanding,gameweek_goals_scored_expanding,gameweek_goals_conceded_expanding,gameweek_saves_expanding,gameweek_threat_expanding,gameweek_xG_expanding,gameweek_xA_expanding,gameweek_xGA_expanding,gameweek_minutes_expanding,gameweek_xPoints_expanding,gameweek_assists_expanding_per90,gameweek_bps_expanding_per90,gameweek_creativity_expanding_per90,event_points_expanding_per90,gameweek_goals_scored_expanding_per90,gameweek_goals_conceded_expanding_per90,gameweek_saves_expanding_per90,gameweek_threat_expanding_per90,gameweek_xG_expanding_per90,gameweek_xA_expanding_per90,gameweek_xGA_expanding_per90,gameweek_minutes_expanding_per90,gameweek_xPoints_expanding_per90,xG_overperformance,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,ep_next,ep_this,form,in_dreamteam,news,news_added,photo,special,squad_number,status,team,team_code,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,corners_and_indirect_freekicks_text,direct_freekicks_text,penalties_text,expected_goals_per_90,saves_per_90,expected_assists_per_90,expected_goal_involvements_per_90,expected_goals_conceded_per_90,goals_conceded_per_90,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90,name,data_retrieved_datetime,region,can_transact,ca

(4380, 227)

In [7]:
if latest_gameweek>0:
    filepath = Path('../data/fpl_df.csv')
elif latest_gameweek==0:
    filepath = Path('../data/fpl_df_preseason_online.csv')
else:
    print('Check latest_gameweek!')

fpl_df = pd.read_csv(filepath, index_col=0, low_memory=False)
fpl_df = fpl_df[fpl_df.season=='25-26']
display(fpl_df.head())
display(fpl_df.shape)

,can_transact,can_select,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,id,in_dreamteam,news,news_added,now_cost,photo,points_per_game,removed,second_name,selected_by_percent,special,squad_number,status,team,team_code,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,region,team_join_date,birth_date,has_temporary_code,opta_code,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,clearances_blocks_interceptions,recoveries,tackles,defensive_contribution,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text,expected_goals_per_90,saves_per_90,expected_assists_per_90,expected_goal_involvements_per_90,expected_goals_conceded_per_90,goals_conceded_per_90,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90,defensive_contribution_per_90,team_name,name,season
0,True,True,NaN,NaN,154561,0,0,0,0,0,1,4.5,NaN,0,David,0.0,1,False,NaN,NaN,55,154561.jpg,3.7,False,Raya Martín,17.7,False,NaN,a,1,3,142,0,0,0,0,0.0,25.8,Raya,200.0,2024-07-04,1995-09-15,False,p154561,3420,0,0,13,34,0,0,0,3,0,86,10,555,755.4,10.7,0.0,76.7,0,0,0,0,38,0.00,0.03,0.03,35.03,37,8,342,11,640,79,178,8,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2.26,0.00,0.00,0.92,0.89,129,4,481,79,68,11,22,4,1.00,0.34,0.00,Arsenal,David Raya Martín,25-26
1,True,True,NaN,NaN,109745,0,0,0,0,0,1,2.8,NaN,0,Kepa,0.0,2,False,NaN,NaN,45,109745.jpg,3.4,False,Arrizabalaga Revuelta,1.0,False,NaN,a,1,3,106,0,0,0,0,0.0,23.6,Arrizabalaga,200.0,2025-07-01,1994-10-03,False,p109745,2790,0,0,8,39,0,0,0,3,0,97,7,406,744.4,10.4,4.0,75.8,0,0,0,0,31,0.06,0.03,0.09,38.83,38,9,344,12,333,1,183,9,NaN,NaN,NaN,NaN,NaN,NaN,0.0,3.13,0.00,0.00,1.25,1.26,372,22,144,51,94,16,198,28,1.00,0.26,0.00,Arsenal,Kepa Arrizabalaga Revuelta,25-26
2,True,True,NaN,NaN,463748,0,0,0,0,0,1,1.5,NaN,0,Karl,0.0,3,False,NaN,NaN,40,463748.jpg,0.0,False,Hein,0.6,False,NaN,a,1,3,0,0,0,0,0,0.0,0.0,Hein,67.0,2020-10-28,2002-04-13,False,p463748,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.00,0.00,0.00,0.00,437,55,427,46,396,31,437,55,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.00,0.00,0.00,0.00,0.00,610,63,58,29,440,55,247,33,0.00,0.00,0.00,Arsenal,Karl Hein,25-26
3,True,True,NaN,NaN,551221,0,0,0,0,0,1,1.5,NaN,0,Tommy,0.0,4,False,NaN,NaN,40,551221.jpg,0.0,False,Setford,0.2,False,NaN,a,1,3,0,0,0,0,0,0.0,0.0,Setford,241.0,2024-07-24,2006-03-13,False,p551221,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.00,0.00,0.00,0.00,471,66,462,57,436,44,471,66,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.00,0.00,0.00,0.00,0.00,653,76,101,42,474,66,451,61,0.00,0.00,0.00,Arsenal,Tommy Setford,25-26
4,True,True,NaN,NaN,226597,0,0,0,0,0,2,4.5,NaN,0,Gabriel,0.0,5,False,NaN,NaN,60,226597.jpg,4.2,False,dos Santos Magalhães,21.4,False,NaN,a,1,3,117,0,0,0,0,0.0,19.5,Gabriel,30.0,2020-09-01,1997-12-19,False,p226597,2363,3,2,10,22,0,0,0,4,0,0,9,459,584.6,208.8,287.0,108.2,0,0,0,159,28,2.70,1.21,3.91,23.09,96,32,167,36,107,12,121,31,NaN,NaN,NaN,NaN,NaN,NaN,0.1,0.00,0.05,0.15,0.88,0.84,98,5,584,219,32,1,16,6,1.07,0.38,6.06,Arsenal,Gabriel dos Santos Magalhães,25-26


(677, 104)

In [8]:
df = fpl_df.groupby('name').last().reset_index()[['id', 'name', 'element_type', 'points_per_game', 'total_points',]]
#df['id'] = df.id.astype(int)
df['games_played'] = np.round(np.where(df['points_per_game']!=0, df['total_points'] / df['points_per_game'], 0),0)
display(df.head())
display(df.shape)

,id,name,element_type,points_per_game,total_points,games_played
0,116,Aaron Hickey,2,0.0,0,0.0
1,674,Aaron Ramsdale,1,0.0,0,0.0
2,211,Aaron Ramsey,3,0.0,0,0.0
3,610,Aaron Wan-Bissaka,2,3.3,118,36.0
4,234,Aarón Anselmino,2,0.0,0,0.0


(677, 6)

In [9]:
# drop duplicate players (some players get new spelling for their name during the season causing duplicates)
duplicate_ids = df.loc[df.id.duplicated(), 'id'].unique()
for id in duplicate_ids:
    ix = df.loc[df.id==id, 'games_played'].idxmin()
    df = df.drop(ix)
display(df.head())
display(df.shape)

,id,name,element_type,points_per_game,total_points,games_played
0,116,Aaron Hickey,2,0.0,0,0.0
1,674,Aaron Ramsdale,1,0.0,0,0.0
2,211,Aaron Ramsey,3,0.0,0,0.0
3,610,Aaron Wan-Bissaka,2,3.3,118,36.0
4,234,Aarón Anselmino,2,0.0,0,0.0


(677, 6)

In [10]:
# merge id info to projections
projections = projections.merge(df[['id', 'name']], on='name', how='left')
projections.head()

,index,first_name,second_name,assists,bonus,bps,clean_sheets,corners_and_indirect_freekicks_order,creativity,creativity_rank,creativity_rank_type,direct_freekicks_order,dreamteam_count,element_type,event_points,goals_conceded,goals_scored,ict_index,ict_index_rank,ict_index_rank_type,influence,influence_rank,influence_rank_type,minutes,now_cost,own_goals,penalties_missed,penalties_order,penalties_saved,points_per_game,red_cards,saves,selected_by_percent,threat,threat_rank,threat_rank_type,total_points,web_name,yellow_cards,team_name,gameweek,season,gameweek_xG,gameweek_xA,gameweek_xGA,gameweek_minutes,team_xG,team_xGA,team_xG_ewm_5,team_xG_ewm_10,team_xG_ewm_20,team_xG_ewm_40,team_xGA_ewm_5,team_xGA_ewm_10,team_xGA_ewm_20,team_xGA_ewm_40,opponent_xG,opponent_xGA,opponent_xG_ewm_5,opponent_xG_ewm_10,opponent_xG_ewm_20,opponent_xG_ewm_40,opponent_xGA_ewm_5,opponent_xGA_ewm_10,opponent_xGA_ewm_20,opponent_xGA_ewm_40,home,gameweek_assists,gameweek_bps,gameweek_creativity,gameweek_goals_scored,gameweek_goals_conceded,gameweek_own_goals,gameweek_penalties_saved,gameweek_red_cards,gameweek_saves,gameweek_threat,gameweek_yellow_cards,gameweek_xPoints,gameweek_assists_ewm_5,gameweek_bps_ewm_5,gameweek_creativity_ewm_5,event_points_ewm_5,gameweek_goals_scored_ewm_5,gameweek_goals_conceded_ewm_5,gameweek_saves_ewm_5,gameweek_threat_ewm_5,gameweek_xG_ewm_5,gameweek_xA_ewm_5,gameweek_xGA_ewm_5,gameweek_minutes_ewm_5,gameweek_xPoints_ewm_5,gameweek_assists_ewm_10,gameweek_bps_ewm_10,gameweek_creativity_ewm_10,event_points_ewm_10,gameweek_goals_scored_ewm_10,gameweek_goals_conceded_ewm_10,gameweek_saves_ewm_10,gameweek_threat_ewm_10,gameweek_xG_ewm_10,gameweek_xA_ewm_10,gameweek_xGA_ewm_10,gameweek_minutes_ewm_10,gameweek_xPoints_ewm_10,gameweek_assists_ewm_20,gameweek_bps_ewm_20,gameweek_creativity_ewm_20,event_points_ewm_20,gameweek_goals_scored_ewm_20,gameweek_goals_conceded_ewm_20,gameweek_saves_ewm_20,gameweek_threat_ewm_20,gameweek_xG_ewm_20,gameweek_xA_ewm_20,gameweek_xGA_ewm_20,gameweek_minutes_ewm_20,gameweek_xPoints_ewm_20,gameweek_assists_ewm_40,gameweek_bps_ewm_40,gameweek_creativity_ewm_40,event_points_ewm_40,gameweek_goals_scored_ewm_40,gameweek_goals_conceded_ewm_40,gameweek_saves_ewm_40,gameweek_threat_ewm_40,gameweek_xG_ewm_40,gameweek_xA_ewm_40,gameweek_xGA_ewm_40,gameweek_minutes_ewm_40,gameweek_xPoints_ewm_40,gameweek_assists_expanding,gameweek_bps_expanding,gameweek_creativity_expanding,event_points_expanding,gameweek_goals_scored_expanding,gameweek_goals_conceded_expanding,gameweek_saves_expanding,gameweek_threat_expanding,gameweek_xG_expanding,gameweek_xA_expanding,gameweek_xGA_expanding,gameweek_minutes_expanding,gameweek_xPoints_expanding,gameweek_assists_expanding_per90,gameweek_bps_expanding_per90,gameweek_creativity_expanding_per90,event_points_expanding_per90,gameweek_goals_scored_expanding_per90,gameweek_goals_conceded_expanding_per90,gameweek_saves_expanding_per90,gameweek_threat_expanding_per90,gameweek_xG_expanding_per90,gameweek_xA_expanding_per90,gameweek_xGA_expanding_per90,gameweek_minutes_expanding_per90,gameweek_xPoints_expanding_per90,xG_overperformance,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,ep_next,ep_this,form,in_dreamteam,news,news_added,photo,special,squad_number,status,team,team_code,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,corners_and_indirect_freekicks_text,direct_freekicks_text,penalties_text,expected_goals_per_90,saves_per_90,expected_assists_per_90,expected_goal_involvements_per_90,expected_goals_conceded_per_90,goals_conceded_per_90,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90,name,data_retrieved_datetime,region,can_transact,ca

In [11]:
# add xMins variable that is needed later
projections['xMins'] = 90

In [12]:
projections[['id', 'expected_points', 'xMins', 'gameweek']]

,id,expected_points,xMins,gameweek
0,116,2.379389,90,1
1,116,2.280249,90,2
2,116,2.826401,90,3
3,116,2.210212,90,4
4,116,2.318986,90,5
...,...,...,...,...
4375,67,3.595683,90,6
4376,67,3.531511,90,7
4377,67,2.837555,90,8
4378,67,3.658122,90,9


In [13]:
projections_pivot = (
    projections
    .pivot_table(
    columns=['gameweek',],
    index='id', 
    values=['expected_points','xMins'], 
    aggfunc='sum'
    )
)

projections_pivot.head()

expected_points                                                    \
gameweek              1         2         3         4         5         6    
id                                                                           
5               3.619536  4.181914  2.948266  3.982273  3.124078  3.223237   
6               3.388043  4.032066  2.634778  3.863696  2.983860  3.035604   
7               2.054400  2.331539  1.788588  2.247675  1.988125  1.879263   
8               3.561734  4.336623  3.032037  4.320905  3.203551  3.197579   
9               2.879057  3.577192  2.173590  3.379791  2.605091  2.504729   

                                                 xMins                      \
gameweek        7         8         9         10    1   2   3   4   5   6    
id                                                                           
5         4.067587  3.802357  3.422718  4.240448    90  90  90  90  90  90   
6         3.917229  3.611503  3.310334  4.055395    90  90  90  90  90  90   
7         2.291850  2.132803  2.095537  2.269224    90  90  90  90  90  90   
8         4.298151  4.015655  3.376563  4.440535    90  90  90  90  90  90   
9         3.491802  3.095773  2.801070  3.576103    90  90  90  90  90  90   

                          
gameweek  7   8   9   10  
id                        
5         90  90  90  90  
6         90  90  90  90  
7         90  90  90  90  
8         90  90  90  90  
9         90  90  90  90

In [14]:
new_cols = []
for col in projections_pivot.columns:
    if col[0] == 'expected_points':
        new_col = str(col[1]) + '_Pts'
        new_cols.append(new_col)
    elif col[0] == 'xMins':
        new_col = str(col[1]) + '_xMins'
        new_cols.append(new_col)

projections_pivot.columns = new_cols
projections_pivot = projections_pivot.reset_index()
projections_pivot = projections_pivot.rename(columns={'id':'ID'})

projections_pivot.head()

,ID,1_Pts,2_Pts,3_Pts,4_Pts,5_Pts,6_Pts,7_Pts,8_Pts,9_Pts,10_Pts,1_xMins,2_xMins,3_xMins,4_xMins,5_xMins,6_xMins,7_xMins,8_xMins,9_xMins,10_xMins
0,5,3.619536,4.181914,2.948266,3.982273,3.124078,3.223237,4.067587,3.802357,3.422718,4.240448,90,90,90,90,90,90,90,90,90,90
1,6,3.388043,4.032066,2.634778,3.863696,2.983860,3.035604,3.917229,3.611503,3.310334,4.055395,90,90,90,90,90,90,90,90,90,90
2,7,2.054400,2.331539,1.788588,2.247675,1.988125,1.879263,2.291850,2.132803,2.095537,2.269224,90,90,90,90,90,90,90,90,90,90
3,8,3.561734,4.336623,3.032037,4.320905,3.203551,3.197579,4.298151,4.015655,3.376563,4.440535,90,90,90,90,90,90,90,90,90,90
4,9,2.879057,3.577192,2.173590,3.379791,2.605091,2.504729,3.491802,3.095773,2.801070,3.576103,90,90,90,90,90,90,90,90,90,90


In [15]:
position_dict = {1:'G', 2:'D', 3:'M', 4:'F'}
df['Pos'] = df['element_type'].map(position_dict)
df.rename(columns={'id':'ID'}, inplace=True)    

In [16]:
projections_pivot = pd.merge(projections_pivot, df[['ID', 'Pos']], on='ID', how='left')
projections_pivot.head()

,ID,1_Pts,2_Pts,3_Pts,4_Pts,5_Pts,6_Pts,7_Pts,8_Pts,9_Pts,10_Pts,1_xMins,2_xMins,3_xMins,4_xMins,5_xMins,6_xMins,7_xMins,8_xMins,9_xMins,10_xMins,Pos
0,5,3.619536,4.181914,2.948266,3.982273,3.124078,3.223237,4.067587,3.802357,3.422718,4.240448,90,90,90,90,90,90,90,90,90,90,D
1,6,3.388043,4.032066,2.634778,3.863696,2.983860,3.035604,3.917229,3.611503,3.310334,4.055395,90,90,90,90,90,90,90,90,90,90,D
2,7,2.054400,2.331539,1.788588,2.247675,1.988125,1.879263,2.291850,2.132803,2.095537,2.269224,90,90,90,90,90,90,90,90,90,90,D
3,8,3.561734,4.336623,3.032037,4.320905,3.203551,3.197579,4.298151,4.015655,3.376563,4.440535,90,90,90,90,90,90,90,90,90,90,D
4,9,2.879057,3.577192,2.173590,3.379791,2.605091,2.504729,3.491802,3.095773,2.801070,3.576103,90,90,90,90,90,90,90,90,90,90,D


In [ ]:
# 25-26 pre-season adjustments for defensive contributions

def_contribution_ids = [111, 257, 291, 506,  24,  85,  87, 129, 167, 173, 241, 272, 273,
       302, 457, 459, 517, 519, 585, 586, 587, 643]

cols = ['1_Pts','2_Pts','3_Pts','4_Pts','5_Pts','6_Pts','7_Pts','8_Pts','9_Pts','10_Pts']

display(projections_pivot.loc[projections_pivot.ID.isin(def_contribution_ids),['ID']+cols])

for id in def_contribution_ids:
    projections_pivot.loc[projections_pivot['ID']==id, cols] += 1

display(projections_pivot.loc[projections_pivot.ID.isin(def_contribution_ids),['ID']+cols])

In [17]:
filepath = Path('../../../fpl-optimization/data/fplreview.csv')
projections_pivot.to_csv(filepath)

### Option to make manual changes, e.g., to xPts (based on injuries)

### adjust points

If model is underestimating e.g. Haaland currently, test how many points more he would need to be getting in order to be drafed to the team in the optimization.

In [ ]:
projections.loc[projections['name'].str.contains('Wood'), 'id'].unique()
#projections.loc[projections['name'].str.contains('Palme'), 'id'].unique()

In [ ]:
player_id = 477
display(projections.loc[projections.id==player_id, 'name'].unique())
display(projections_pivot[projections_pivot.ID==player_id])

In [ ]:
pts_cols = [col for col in projections_pivot.columns if 'Pts' in col]
projections_pivot.loc[projections_pivot.ID==player_id, pts_cols].sum(axis=1) / 10

In [ ]:
gameweeks = np.arange(latest_gameweek+1,latest_gameweek+11)
add_constant = -0.5

for i in range(len(gameweeks)):
    projections_pivot.loc[projections_pivot.ID==player_id, f'{gameweeks[i]}_Pts'] = \
        projections_pivot.loc[projections_pivot.ID==player_id, f'{gameweeks[i]}_Pts'] + add_constant

display(projections_pivot[projections_pivot.ID==player_id])

### Adjust points for unavailable players or players with uncertain availability

In [18]:
def adjust_points_via_weights(player_id, gameweeks, weights):
    '''Adjust projected player points via gameweek specific weights indicating probability of playing.

    Input:
    player_id (int): Player ID number.
    gameweeks (list): List of gameweek numbers to be adjusted.
    weights (list): Weights for adjusting points for each gameweek given in parameter gameweeks.
    '''
    if len(projections.loc[projections.id==player_id]) > 0:
        player_name = projections.loc[projections.id==player_id, 'name'].unique().item()
        print(f'Name: {player_name}')
        print('Point predictions before adjustment:')
        display(projections_pivot[projections_pivot.ID==player_id])

        # adjust points via probability of playing
        for i in range(len(gameweeks)):
            projections_pivot.loc[projections_pivot.ID==player_id, f'{gameweeks[i]}_Pts'] = \
                weights[i]*projections_pivot.loc[projections_pivot.ID==player_id, f'{gameweeks[i]}_Pts']
            
        print('Point predictions after adjustment:')
        display(projections_pivot[projections_pivot.ID==player_id])
    else:
        print(f'No player with ID {player_id} found in projections!')

### Automatic adjustments

In [19]:
# fetch online data
fpl_online_data = json.loads(requests.get('https://fantasy.premierleague.com/api/bootstrap-static/').text)
fpl_online_df = pd.DataFrame(fpl_online_data['elements'])

In [20]:
# get play probabilities for players with injuries
play_probabilities = []
for _, row in fpl_online_df.iterrows():
    news = row['news']
    news_splitted = news.split('-')
    if len(news_splitted) > 1:
        first_3_chars = news_splitted[1].strip()[0:3]
        if first_3_chars == '25%':
            play_probabilities.append(0.25)
        elif first_3_chars == '50%':
            play_probabilities.append(0.5)
        elif first_3_chars == '75%':
            play_probabilities.append(0.75)
        else:
            play_probabilities.append(1.0)
    else:
        play_probabilities.append(1.0)
fpl_online_df['play_probability'] = play_probabilities

In [21]:
# adjust player points based on play probabilities
# only include players with minutes played (which have existing projections)
prob25_ids = fpl_online_df.loc[(fpl_online_df.play_probability==0.25) & (fpl_online_df.minutes>0), 'id'].values
prob50_ids = fpl_online_df.loc[(fpl_online_df.play_probability==0.5) & (fpl_online_df.minutes>0), 'id'].values
prob75_ids = fpl_online_df.loc[(fpl_online_df.play_probability==0.75) & (fpl_online_df.minutes>0), 'id'].values

for player_id in prob25_ids:
    if latest_gameweek<=35:
        adjust_points_via_weights(player_id=player_id, gameweeks=[latest_gameweek+1, latest_gameweek+2, latest_gameweek+3], weights=[0.25,0.5,0.75])
    elif latest_gameweek==36:
        adjust_points_via_weights(player_id=player_id, gameweeks=[latest_gameweek+1, latest_gameweek+2], weights=[0.25,0.5])
    elif latest_gameweek==37:
        adjust_points_via_weights(player_id=player_id, gameweeks=[latest_gameweek+1], weights=[0.25])
    else:
        print('Check latest_gameweek!')

for player_id in prob50_ids:
    if latest_gameweek<=36:
        adjust_points_via_weights(player_id=player_id, gameweeks=[latest_gameweek+1, latest_gameweek+2], weights=[0.5,0.75])
    elif latest_gameweek==37:
        adjust_points_via_weights(player_id=player_id, gameweeks=[latest_gameweek+1], weights=[0.5])
    else:
        print('Check latest_gameweek!')

for player_id in prob75_ids:
    adjust_points_via_weights(player_id=player_id, gameweeks=[latest_gameweek+1], weights=[0.75])

Name: Solly March
Point predictions before adjustment:


,ID,1_Pts,2_Pts,3_Pts,4_Pts,5_Pts,6_Pts,7_Pts,8_Pts,9_Pts,10_Pts,1_xMins,2_xMins,3_xMins,4_xMins,5_xMins,6_xMins,7_xMins,8_xMins,9_xMins,10_xMins,Pos
108,159,2.137487,2.090773,1.878282,1.971686,2.132145,1.91615,2.174815,1.956284,2.01215,2.293316,90,90,90,90,90,90,90,90,90,90,M


Point predictions after adjustment:


,ID,1_Pts,2_Pts,3_Pts,4_Pts,5_Pts,6_Pts,7_Pts,8_Pts,9_Pts,10_Pts,1_xMins,2_xMins,3_xMins,4_xMins,5_xMins,6_xMins,7_xMins,8_xMins,9_xMins,10_xMins,Pos
108,159,0.534372,1.045387,1.408712,1.971686,2.132145,1.91615,2.174815,1.956284,2.01215,2.293316,90,90,90,90,90,90,90,90,90,90,M


No player with ID 229 found in projections!
Name: Roméo Lavia
Point predictions before adjustment:


...

,ID,1_Pts,2_Pts,3_Pts,4_Pts,5_Pts,6_Pts,7_Pts,8_Pts,9_Pts,10_Pts,1_xMins,2_xMins,3_xMins,4_xMins,5_xMins,6_xMins,7_xMins,8_xMins,9_xMins,10_xMins,Pos
161,252,1.334913,1.710785,1.696522,1.550179,1.730425,1.694702,1.629684,1.786417,1.555088,1.770921,90,90,90,90,90,90,90,90,90,90,F


Name: Destiny Udogie
Point predictions before adjustment:


,ID,1_Pts,2_Pts,3_Pts,4_Pts,5_Pts,6_Pts,7_Pts,8_Pts,9_Pts,10_Pts,1_xMins,2_xMins,3_xMins,4_xMins,5_xMins,6_xMins,7_xMins,8_xMins,9_xMins,10_xMins,Pos
368,574,2.41985,1.674334,2.055144,2.189983,1.832221,2.342488,2.350578,2.058795,2.065702,1.977627,90,90,90,90,90,90,90,90,90,90,D


Point predictions after adjustment:


,ID,1_Pts,2_Pts,3_Pts,4_Pts,5_Pts,6_Pts,7_Pts,8_Pts,9_Pts,10_Pts,1_xMins,2_xMins,3_xMins,4_xMins,5_xMins,6_xMins,7_xMins,8_xMins,9_xMins,10_xMins,Pos
368,574,1.814887,1.674334,2.055144,2.189983,1.832221,2.342488,2.350578,2.058795,2.065702,1.977627,90,90,90,90,90,90,90,90,90,90,D


In [22]:
filepath = Path('../../../fpl-optimization/data/fplreview.csv')
projections_pivot.to_csv(filepath)

### Manual adjustments

In [ ]:
# Check player_id
projections.loc[projections['name'].str.contains('Pir'), 'id'].unique()

In [ ]:
player_id = 348
display(projections.loc[projections.id==player_id, 'name'].unique())

In [ ]:
adjust_points_via_weights(player_id=4, gameweeks=[38], weights=[0.75])

In [ ]:
adjust_points_via_weights(player_id=23, gameweeks=[38], weights=[0.5])

In [ ]:
adjust_points_via_weights(player_id=129, gameweeks=[34,35,36], weights=[0,0,0])

In [ ]:
adjust_points_via_weights(player_id=521, gameweeks=[34,35,36,37,38], weights=[0,0,0,0,0])

In [ ]:
adjust_points_via_weights(player_id=477, gameweeks=[34,35,36,37,38], weights=[0.25,0.25,0.25,0.25,0.25])

In [ ]:
adjust_points_via_weights(player_id=554, gameweeks=[34, 35], weights=[0.5, 0.75])

In [ ]:
adjust_points_via_weights(player_id=473, gameweeks=[31,32,33,34,35,36], weights=[0.25,0.25,0.25,0.25,0.25,0.25])

In [ ]:
adjust_points_via_weights(player_id=398, gameweeks=[30,31], weights=[0.5,0.75])

In [ ]:
adjust_points_via_weights(player_id=447, gameweeks=[32], weights=[0.75])

In [ ]:
adjust_points_via_weights(player_id=541, gameweeks=[30,31], weights=[0,0])

In [ ]:
projections.shape

In [ ]:
projections.head()

In [ ]:
# drop cheap goalkeepers
projections = projections[~((projections.now_cost<42) & (projections.element_type==1))].copy()

In [ ]:
filepath = Path('../../../fpl-optimization/data/fplreview.csv')
projections_pivot.to_csv(filepath)

# Other stuff

In [ ]:
# check ids
projections.loc[projections['name'].str.contains('Wissa'), ['id','name']]

In [ ]:
projections.loc[projections.id==218, ['name','team_name']]

In [ ]:
# locked
projections.loc[projections.id.isin([470, 348, 263]), ['id','name']].groupby('id').first()

In [23]:
# banned players from solver 
projections.loc[projections.id.isin([20, 379, 138, 610, 119, 19, 8, 363, 135, 668, 24, 499, 30, 111, 11]), ['id','name']].groupby('id').first()

,name
id,
8,Jurriën Timber
11,Benjamin White
19,Gabriel Martinelli Silva
20,Leandro Trossard
24,Christian Nørgaard
30,Kai Havertz
111,Ethan Pinnock
119,Bryan Mbeumo
135,Yoane Wissa


In [24]:
projections[projections.id==498]

,index,first_name,second_name,assists,bonus,bps,clean_sheets,corners_and_indirect_freekicks_order,creativity,creativity_rank,creativity_rank_type,direct_freekicks_order,dreamteam_count,element_type,event_points,goals_conceded,goals_scored,ict_index,ict_index_rank,ict_index_rank_type,influence,influence_rank,influence_rank_type,minutes,now_cost,own_goals,penalties_missed,penalties_order,penalties_saved,points_per_game,red_cards,saves,selected_by_percent,threat,threat_rank,threat_rank_type,total_points,web_name,yellow_cards,team_name,gameweek,season,gameweek_xG,gameweek_xA,gameweek_xGA,gameweek_minutes,team_xG,team_xGA,team_xG_ewm_5,team_xG_ewm_10,team_xG_ewm_20,team_xG_ewm_40,team_xGA_ewm_5,team_xGA_ewm_10,team_xGA_ewm_20,team_xGA_ewm_40,opponent_xG,opponent_xGA,opponent_xG_ewm_5,opponent_xG_ewm_10,opponent_xG_ewm_20,opponent_xG_ewm_40,opponent_xGA_ewm_5,opponent_xGA_ewm_10,opponent_xGA_ewm_20,opponent_xGA_ewm_40,home,gameweek_assists,gameweek_bps,gameweek_creativity,gameweek_goals_scored,gameweek_goals_conceded,gameweek_own_goals,gameweek_penalties_saved,gameweek_red_cards,gameweek_saves,gameweek_threat,gameweek_yellow_cards,gameweek_xPoints,gameweek_assists_ewm_5,gameweek_bps_ewm_5,gameweek_creativity_ewm_5,event_points_ewm_5,gameweek_goals_scored_ewm_5,gameweek_goals_conceded_ewm_5,gameweek_saves_ewm_5,gameweek_threat_ewm_5,gameweek_xG_ewm_5,gameweek_xA_ewm_5,gameweek_xGA_ewm_5,gameweek_minutes_ewm_5,gameweek_xPoints_ewm_5,gameweek_assists_ewm_10,gameweek_bps_ewm_10,gameweek_creativity_ewm_10,event_points_ewm_10,gameweek_goals_scored_ewm_10,gameweek_goals_conceded_ewm_10,gameweek_saves_ewm_10,gameweek_threat_ewm_10,gameweek_xG_ewm_10,gameweek_xA_ewm_10,gameweek_xGA_ewm_10,gameweek_minutes_ewm_10,gameweek_xPoints_ewm_10,gameweek_assists_ewm_20,gameweek_bps_ewm_20,gameweek_creativity_ewm_20,event_points_ewm_20,gameweek_goals_scored_ewm_20,gameweek_goals_conceded_ewm_20,gameweek_saves_ewm_20,gameweek_threat_ewm_20,gameweek_xG_ewm_20,gameweek_xA_ewm_20,gameweek_xGA_ewm_20,gameweek_minutes_ewm_20,gameweek_xPoints_ewm_20,gameweek_assists_ewm_40,gameweek_bps_ewm_40,gameweek_creativity_ewm_40,event_points_ewm_40,gameweek_goals_scored_ewm_40,gameweek_goals_conceded_ewm_40,gameweek_saves_ewm_40,gameweek_threat_ewm_40,gameweek_xG_ewm_40,gameweek_xA_ewm_40,gameweek_xGA_ewm_40,gameweek_minutes_ewm_40,gameweek_xPoints_ewm_40,gameweek_assists_expanding,gameweek_bps_expanding,gameweek_creativity_expanding,event_points_expanding,gameweek_goals_scored_expanding,gameweek_goals_conceded_expanding,gameweek_saves_expanding,gameweek_threat_expanding,gameweek_xG_expanding,gameweek_xA_expanding,gameweek_xGA_expanding,gameweek_minutes_expanding,gameweek_xPoints_expanding,gameweek_assists_expanding_per90,gameweek_bps_expanding_per90,gameweek_creativity_expanding_per90,event_points_expanding_per90,gameweek_goals_scored_expanding_per90,gameweek_goals_conceded_expanding_per90,gameweek_saves_expanding_per90,gameweek_threat_expanding_per90,gameweek_xG_expanding_per90,gameweek_xA_expanding_per90,gameweek_xGA_expanding_per90,gameweek_minutes_expanding_per90,gameweek_xPoints_expanding_per90,xG_overperformance,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,ep_next,ep_this,form,in_dreamteam,news,news_added,photo,special,squad_number,status,team,team_code,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,corners_and_indirect_freekicks_text,direct_freekicks_text,penalties_text,expected_goals_per_90,saves_per_90,expected_assists_per_90,expected_goal_involvements_per_90,expected_goals_conceded_per_90,goals_conceded_per_90,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90,name,data_retrieved_datetime,region,can_transact,ca

In [ ]:
projections_pivot.columns